# Retrieve research results (validation)

To inspect the reported research results you must install the following julia packages:
**JLD**, **PGFPlotsX**, **Plots**, **LaTeXStrings**, **CSV**, **DataFrames**

Notational conventions: prefixes **m** and **v** denote mean and variance of the random variables respectively.

For a sake of brevity, throughout the code we use $x$ and $z$ instead of $x^{(1)}$ and $x^{(2)}$ respectively.

In [1]:
using JLD, Plots, PGFPlotsX, SparseArrays, CSV, DataFrames, LaTeXStrings
pgfplotsx()
push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepgfplotslibrary{fillbetween}")

# df = CSV.File("data/AAPL.csv") |> DataFrame
# AAPL_stocks = df[!, :Open];
df = CSV.File("data/capr_prl_one_subject.csv") |> DataFrame
capr_one_subject = df[!, :reward_tally];

┌ Warning: backend `PGFPlotsX` is not installed.
└ @ Plots C:\Users\mkpap\.julia\packages\Plots\sxUvK\src\backends.jl:43


In [2]:
shgf3l_results = JLD.load("dump/results_3shgf_stocks_mixture.jld")
shgf2l_results = JLD.load("dump/results_2shgf_stocks_mixture.jld")
shgf2l3c_results = JLD.load("dump/results_2l3cshgf_stocks_mixture.jld")
hgf2l_results  = JLD.load("dump/results_hgf_stocks_mixture.jld");

In [ ]:
fe_shgf3l = shgf3l_results["results"]["fe"][2:end]
fe_shgf2l = shgf2l_results["results"]["fe"][2:end]
fe_shgf2l3c = shgf2l3c_results["results"]["fe"][2:end]
fe_hgf2l  = hgf2l_results["results"]["fe"][2:end]

fe_max_iter = 40
free_energy_axis = @pgf Axis({xlabel="iteration",
           ylabel="free-enery [nats]",
           legend_pos = "north east",
           grid = "major",
           mark_options = {scale=0.3}, xmin=-5.0, xmax=45,
    },
    Plot({mark = "square*", "blue"},Coordinates(collect(1:fe_max_iter),fe_shgf3l[1:fe_max_iter])),
    LegendEntry("3L-SHGF-2"),
    Plot({mark = "triangle*", "red"},Coordinates(collect(1:fe_max_iter),fe_shgf2l[1:fe_max_iter])),
    LegendEntry("2L-SHGF-2"),
    Plot({mark = "square", "green"},Coordinates(collect(1:fe_max_iter),fe_shgf2l3c[1:fe_max_iter])),
    LegendEntry("2L-SHGF-3"),
    Plot({mark = "o", "black"},Coordinates(collect(1:fe_max_iter),fe_hgf2l[1:fe_max_iter])),
    LegendEntry("2L-HGF"),
)
pgfsave("figures/validation_results_free_energy.tikz", free_energy_axis)

In [ ]:
mz_shgf3l = shgf3l_results["results"]["mz1"]
mz_shgf2l = shgf2l_results["results"]["mz"]
mz_shgf2l3c = shgf2l3c_results["results"]["mz"]
mz_hgf2l  = hgf2l_results["results"]["mz"]
vz_shgf3l = shgf3l_results["results"]["vz1"]
vz_shgf2l = shgf2l_results["results"]["vz"]
vz_shgf2l3c = shgf2l3c_results["results"]["vz"]
vz_hgf2l  = hgf2l_results["results"]["vz"]
n_samples = length(AAPL_stocks)
axis3 = @pgf Axis({xlabel=L"t",
           ylabel=L"x_t^{(2)}",
        legend_pos = "north west",
        mark_options = {scale=0.3},
        grid="major",
        style = {thick}
    },
    
    Plot({no_marks,color="blue"},Coordinates(collect(1:n_samples), mz_shgf3l)),
    LegendEntry("3L-SHGF-2"),
    Plot({no_marks,color="red"}, Coordinates(collect(1:n_samples), mz_shgf2l)),
    LegendEntry("2L-SHGF-2"),
    Plot({no_marks,color="green"},Coordinates(collect(1:n_samples), mz_shgf2l3c)),
    LegendEntry("2L-SHGF-3"),
    Plot({no_marks,color="black"},Coordinates(collect(1:n_samples), mz_hgf2l)),
    LegendEntry("2L-HGF"),
    
    Plot({ "name path=f1", no_marks,color="blue",opacity=0.2 }, Coordinates(collect(1:n_samples), mz_shgf3l .+  sqrt.(vz_shgf3l))),
    Plot({ "name path=g1", no_marks, color="blue",opacity=0.2}, Coordinates(collect(1:n_samples), mz_shgf3l .-  sqrt.(vz_shgf3l))),
    Plot({ thick, color = "blue", fill = "blue", opacity = 0.2 },
              raw"fill between [of=f1 and g1]"),
    
    Plot({ "name path=f2", no_marks,color="red",opacity=0.2 }, Coordinates(collect(1:n_samples), mz_shgf2l .+  sqrt.(vz_shgf2l))),
    Plot({ "name path=g2", no_marks, color="red",opacity=0.2}, Coordinates(collect(1:n_samples), mz_shgf2l .-  sqrt.(vz_shgf2l))),
    Plot({ thick, color = "blue", fill = "red", opacity = 0.2 },
               raw"fill between [of=f2 and g2]"),
    
    
    Plot({ "name path=f3", no_marks,color="green",opacity=0.2 }, Coordinates(collect(1:n_samples), mz_shgf2l3c .+  sqrt.(vz_shgf2l3c))),
    Plot({ "name path=g3", no_marks, color="green",opacity=0.2}, Coordinates(collect(1:n_samples), mz_shgf2l3c .-  sqrt.(vz_shgf2l3c))),
    Plot({ thick, color = "green", fill = "green", opacity = 0.2 },
               raw"fill between [of=f3 and g3]"),
    
   
    Plot({ "name path=f4", no_marks,color="black",opacity=0.2 }, Coordinates(collect(1:n_samples), mz_hgf2l .+  sqrt.(vz_hgf2l))),
    Plot({ "name path=g4", no_marks, color="black",opacity=0.2}, Coordinates(collect(1:n_samples), mz_hgf2l .-  sqrt.(vz_hgf2l))),
    Plot({ thick, color = "blue", fill = "black", opacity = 0.2 },
            raw"fill between [of=f4 and g4]"),
)

pgfsave("figures/validation_results_vol.tikz", axis3)

In [ ]:
mx_shgf3l = ["mx1"]
mx_shgf2l = shgf2l_results["results"]["mx"]
mx_hgf2l  = hgf2l_results["results"]["mx"]
vx_shgf3l = shgf3l_results["results"]["vx"]
vx_shgf2l = shgf2l_results["results"]["vx"]
vx_hgf2l  = hgf2l_results["results"]["vx"];

In [ ]:
ms = shgf2l_results["results"]["ms"]
categories = [x[2] for x in findmax.(ms)]
maxup = maximum(AAPL_stocks) + 3.0
mindown = minimum(AAPL_stocks) - 3.0
axis4 = @pgf Axis({xlabel=L"t",
           ylabel=L"x_t^{(1)}",
        legend_pos = "north west",
        mark_options = {scale=0.3},
        grid="major",
        style = {thick}
    },
    Plot({no_marks,color="magenta"},Coordinates(collect(1:n_samples), mx_shgf2l)),
    LegendEntry("2L-SHGF"),
    Plot({ "name path=f", no_marks,color="magenta",opacity=0.2 }, Coordinates(collect(1:n_samples), mx_shgf2l .+  3*sqrt.(vx_shgf2l))),
    Plot({ "name path=g", no_marks, color="magenta",opacity=0.2}, Coordinates(collect(1:n_samples), mx_shgf2l .-  3*sqrt.(vx_shgf2l))),
    Plot({ thick, color = "magenta", fill = "magenta", opacity = 0.2 },
               raw"fill between [of=f and g]"),
    Plot({only_marks,color="black",opacity=0.3}, Coordinates(collect(1:n_samples),AAPL_stocks)),
    Plot(
       {only_marks, scatter, scatter_src = "explicit"},
       Table(
           {x = "x", y = "y", meta = "col"},
            x = collect(1:n_samples), y = mindown*ones(n_samples), col = categories
       ),
    ),
)

#pgfsave("figures/validation_prices2.tikz", axis4)

In [ ]:
mω = shgf3l_results["results"]["mω2"]